# Building the stochadex

**Mathematical formalism**

Ideally, the stochadex sampler should be designed to try and maintain a balance between performance and flexibility of utilisation (it's meant to be a general stochastic modelling tool after all!). But before we dive into the software, we need to define the mathematical approach we're going to take. It seems reasonable to start with writing down the following formula which describes iterating some arbitrary process forward in time (by one finite step) and adding a new row each to some matrices $S' \rightarrow S$ and $X' \rightarrow X$

$$
\begin{align}
& X^i_{t+\delta t} = F^i_{t+\delta t}(X',t) + S^i_{t+\delta t}(X',S',t),
\end{align}
$$

where to explain things in a bit more detail, let's define

- $\delta t$ as the size of the next timestep
- $i$ as an index for the dimensions of the 'state' space
- $t$ as the current time which may either correspond to the index for some arbitrary stepping process (where $\delta t = 1$), or the explicit time variable for a discrete-time process or some discrete approximation to a continuous-time process
- $X$ as the next version of $X'$ after one timestep (and hence one new row has been added)
- $S$ as the next version of $S'$ (similarly to $X$ and $X'$)
- $F^i_{t+\delta t}(X', t)$ as the latest element of some matrix which very generally can depend on $X'$, $t$ and $\delta t$
- $S^i_{t+\delta t}(X', S', t)$ as the recorded values of a stochastic process defined over the dimensions of the state space indexed by $i$ which may either vary with continuous sample paths in time or not, and which very generally can depend on $X'$, $S'$, $t$ and $\delta t$.

<img src="images/stochadex-formalism-math-diag.png" alt="Drawing" style="width: 700px;"/>

So the idea here is to iterate the matrices $X$ and $S$ forward in time by a row, and use their previous versions ($X'$ and $S'$) as entire matrix inputs into functions which populate the elements of their latest rows. Note that I've avoided using the time variable $t$ and its step size $\delta t$ (reducing the formalism to that of an arbitrary stepping process) to try and make the diagram above more clear, but, more generally, these values may be present in the calculations of new matrix rows.

Why go to all this trouble of storing matrix inputs for previous values of the same process? For a large class of stochastic processes this memory of past values is essential to consistently construct the sample paths moving forward. This is true in particular for _non-Markovian_ phenomena (see, e.g., [Markov chains](https://en.wikipedia.org/wiki/Markov_chain) to get a sense of their antithesis), where the latest values don't just depend on the immediately previous ones but can depend on values which occured much earlier in the process.


In [36]:
from typing import Tuple, List, Optional
import numpy as np
import scipy.special as sps
import plotly.graph_objects as go
import plotly.offline as po

po.init_notebook_mode(connected=True)

def get_next_matrix_row(
    t: float,
    delta_t: float,
    X_prime: np.ndarray,
    S_prime: np.ndarray,
    F: callable,
    S: callable,
    other_noise_prime: Optional[np.ndarray] = None,
    other_noise_iterate: Optional[callable] = None,
) -> Tuple[np.ndarray, np.ndarray]:
    if other_noise_prime is not None:
        other = other_noise_iterate(other_noise_prime)
    else:
        other = None
    s = S(X_prime, S_prime, t, delta_t, other)
    x = F(X_prime, t, delta_t) + s
    return x, s


def generate_x_time_series(
    t_period: float,
    delta_t: float,
    X_init: np.ndarray,
    S_init: np.ndarray,
    F: callable,
    S: callable,
    other_noise_init: Optional[np.ndarray] = None,
    other_noise_iterate: Optional[callable] = None,
) -> List[np.ndarray]:
    x_time_series = []
    X_current, S_current, other_noise_current = X_init, S_init, other_noise_init
    t = 0.0
    while t < t_period:
        x, s = get_next_matrix_row(
            t=t, 
            delta_t=delta_t, 
            X_prime=X_current, 
            S_prime=S_current, 
            F=F,
            S=S,
            other_noise_prime=other_noise_current,
            other_noise_iterate=other_noise_iterate,
        )
        x_time_series.append(x)
        X_current = np.roll(X_current, 1, axis=0)
        S_current = np.roll(S_current, 1, axis=0)
        X_current[0] = x
        S_current[0] = s
        t += delta_t
    return x_time_series


def plot_x_time_series(
    t_period: float,
    delta_t: float,
    X_init: np.ndarray,
    S_init: np.ndarray,
    F: callable,
    S: callable,
    filename: str,
    trace_names: List[str],
    other_noise_init: Optional[np.ndarray] = None,
    other_noise_iterate: Optional[callable] = None,
):
    x_time_series = generate_x_time_series(
        t_period=t_period, 
        delta_t=delta_t, 
        X_init=X_init, 
        S_init=S_init,
        F=F, 
        S=S,
        other_noise_init=other_noise_init,
        other_noise_iterate=other_noise_iterate,
    )
    fig = go.Figure()
    for i in range(len(x_time_series[0])):
        fig.add_trace(
            go.Scatter(
                x=np.arange(0, t_period, delta_t), 
                y=np.asarray(x_time_series).T[i],
                name=trace_names[i],
            )
        )
    fig.update_layout(showlegend=False)
    po.iplot(fig, filename=filename)

**Flavours of noise with continuous sample paths**

For **Wiener process noise**, adopting the [Itô interpretation](https://en.wikipedia.org/wiki/It%C3%B4_calculus) in this section, we can define $W^i_{t}$ is a sample from a Wiener process for each of the state dimensions indexed by $i$ and our formalism becomes

$$
\begin{align}
& X^i_{t+\delta t} = F^i_{t+\delta t}(X', t) + \textcolor{red}{W^i_{t+\delta t}-W^i_{t}}.
\end{align}
$$

In [23]:
plot_x_time_series(
    10.0,
    0.1,
    np.zeros((2, 3)),
    np.zeros((2, 3)), 
    lambda X_prime, t, delta_t: (
        X_prime[0] - delta_t * np.asarray([1.0, 2.0, 3.0]) * (
            X_prime[0] - np.asarray([3.0, 5.0, 2.0])
        )
    ),
    lambda X_prime, S_prime, t, delta_t, other: (
        np.sqrt(delta_t) * np.random.normal(
            np.zeros(3),
            np.asarray([0.5, 1.0, 0.5]), 
            size=3,
        )
    ), 
    "images/wiener_noise.html",
    [
        "Ornstein-Uhlenbeck (1.0, 3.0, 0.5)",
        "Ornstein-Uhlenbeck (2.0, 5.0, 1.0)",
        "Ornstein-Uhlenbeck (3.0, 2.0, 0.5)",
    ]
)

Other interpretations of the noise are less immediately compatible with our formalism as it is currently written, e.g., [Stratonovich](https://en.wikipedia.org/wiki/Stratonovich_integral) or others within the $\alpha$-family, but it seems less necessary to complicate the details of this section further, so we'll just cover these extensions at the software implementation level. Note also that we may also allow for correlations between the noises in different dimensions. 

For **Geometric Brownian motion noise**, we simply have

$$
\begin{align}
& X^i_{t+\delta t} = F^i_{t+\delta t}(X', t) + \textcolor{red}{X^i_{t}(W^i_{t+\delta t}-W^i_{t})}.
\end{align}
$$

In [25]:
plot_x_time_series(
    10.0,
    0.1,
    np.zeros((2, 3)), 
    np.zeros((2, 3)), 
    lambda X_prime, t, delta_t: (
        X_prime[0] - delta_t * np.asarray([1.0, 2.0, 3.0]) * (
            X_prime[0] - np.asarray([3.0, 5.0, 2.0])
        )
    ), 
    lambda X_prime, S_prime, t, delta_t, other: (
        np.sqrt(delta_t) * X_prime[0] * np.random.normal(
            np.zeros(3), 
            np.asarray([0.5, 1.0, 0.5]), 
            size=3,
        )
    ), 
    "images/gbm_noise.html",
    [
        "GBM with drift (1.0, 3.0, 0.5)",
        "GBM with drift (2.0, 5.0, 1.0)",
        "GBM with drift (3.0, 2.0, 0.5)",
    ]
)

And say, e.g., **fractional Brownian motion noise**, where $B^i_{t}({H_i})$ is a sample from a fractional Brownian motion process with Hurst exponent $H_i$ for each of the state dimensions indexed by $i$, we simply substitute again

$$
\begin{align}
& X^i_{t+\delta t} = F^i_{t+\delta t}(X', t) + \textcolor{red}{B^i_{t+\delta t}({H_i})-B^i_{t}({H_i})}.
\end{align}
$$

In [44]:
# note method 2 of simulation here: https://en.wikipedia.org/wiki/Fractional_Brownian_motion
t_period = 10.0
t_step = 0.1

def K(H: float, t: np.ndarray, s: np.ndarray) -> np.ndarray:
    output = (
        ((t-s) ** (H - 0.5)) 
        * sps.hyp2f1(H - 0.5, 0.5 - H, H + 0.5, 1.0 - t/s)
        / sps.gamma(H + 0.5)
    )
    output[s==0] = 1.0
    return output

def brownian_motion_iterate(previous: np.ndarray) -> np.ndarray:
    new = np.roll(previous, 1, axis=0)
    new[0] = np.sqrt(t_step) * np.random.normal(
        np.zeros(3), 
        np.asarray([0.5, 1.0, 0.5]), 
        size=3,
    )
    return new

def fractional_brownian_motion( 
    X_prime: np.ndarray, 
    S_prime: np.ndarray,
    t: float, 
    delta_t: float,
    other: np.ndarray,
) -> np.ndarray:
    fBm_vector = -S_prime[0]
    n = int(np.round(t / delta_t, 1))
    H_values = [0.8, 0.7, 0.3]
    for i in range(3):
        if t == 0:
            fBm_vector[i] = other[0, i]
            continue
        fBm_vector[i] += np.sum(
            K(H_values[i], t, delta_t * np.arange(0, n)) * other[:n, i]
        ) # 'other' is the Brownian motion current windowed history
    return fBm_vector

plot_x_time_series(
    t_period,
    t_step, 
    np.zeros((2, 3)),
    np.zeros((2, 3)), 
    lambda X_prime, t, delta_t: (
        X_prime[0] - delta_t * np.asarray([1.0, 2.0, 3.0]) * (
            X_prime[0] - np.asarray([3.0, 5.0, 2.0])
        )
    ),
    fractional_brownian_motion, 
    "images/fbm_noise.html",
    [
        "fBM with drift (1.0, 3.0, 0.5, H=0.8)",
        "fBM with drift (2.0, 5.0, 1.0, H=0.7)",
        "fBM with drift (3.0, 2.0, 0.5, H=0.3)",
    ],
    other_noise_init=np.zeros((int(t_period / t_step) + 1, 3)),
    other_noise_iterate=brownian_motion_iterate,
)

/tmp/ipykernel_5870/1983031594.py:8: RuntimeWarning:

divide by zero encountered in true_divide



**Generalised continuous noises** would take the form

$$
\begin{align}
& X^i_{t+\delta t} = F^i_{t+\delta t}(X', t) + \textcolor{red}{g^i_{t+\delta t}(X', W^i_{t+\delta t}-W^i_t, \dots)},
\end{align}
$$

where $g^i_{t+\delta t}(X', W^i_{t+\delta t}-W^i_t, \dots)$ is some continuous function of its arguments which can be expanded out with [Itôs Lemma](https://en.wikipedia.org/wiki/It%C3%B4%27s_lemma).

**Flavours of noise with discontinuous sample paths**

**Jump process noises** generally could take the form

$$
\begin{align}
& X^i_{t+\delta t} = F^i_{t+\delta t}(X', t) + \textcolor{red}{J^i_{t+\delta t}(X', \dots )},
\end{align}
$$

where $J^i_{t+\delta t}(X', \dots )$ are samples from some arbitrary jump process (e.g., compound Poisson) which could generally depend on a variety of inputs, including $X'$. 

**Poisson process noises** would generally take the form

$$
\begin{align}
& X^i_{t+\delta t} = F^i_{t+\delta t}(X', t) + \textcolor{red}{N^i_{t+\delta t}({\lambda_i})-N^i_{t}({\lambda_i})},
\end{align}
$$

where $N^i_{t}({\lambda_i})$ is a sample from a Poisson process with rate $\lambda_i$ for each of the state dimensions indexed by $i$. Note that we may also allow for correlations between the noises in different dimensions.

**Time-inhomogeneous Poisson process noises** would generally take the form

$$
\begin{align}
& X^i_{t+\delta t} = F^i_{t+\delta t}(X', t) + \textcolor{red}{N^i_{t+\delta t}({\lambda^i_{t+\delta t}})-N^i_{t}({\lambda^i_t})},
\end{align}
$$

where $\lambda^i_{t}$ is a deterministically-varying rate for each of the state dimensions indexed by $i$.

**Cox (doubly-stochastic) process noises** would generally take the form

$$
\begin{align}
& X^i_{t+\delta t} = F^i_{t+\delta t}(X', t) + \textcolor{red}{N^i_{t+\delta t}({\Lambda^i_{t+\delta t}})-N^i_{t}({\Lambda^i_{t}})},
\end{align}
$$

where the rate $\Lambda^i_{t}$ is now a sample from some continuous-time stochastic process (in the positive-only domain) for each of the state dimensions indexed by $i$.

**Self-exciting process noises** would generally take the form

$$
\begin{align}
& X^i_{t+\delta t} = F^i_{t+\delta t}(X', t) + \textcolor{red}{N^i_{t+\delta t}[{\cal I}^i_{t+\delta t}(N', \dots)]-N^i_{t}[{\cal I}^i_{t}(N'', \dots)]},
\end{align}
$$

where the stochastic rate ${\cal I}^i_{t}(N', \dots)$ now depends on the history of $N'$ explicitly (amongst other potential inputs - see, e.g., [Hawkes processes](https://en.wikipedia.org/wiki/Hawkes_process)) for each of the state dimensions indexed by $i$.

**Generalised probabilistic discrete state transitions** would take the form

$$
\begin{align}
& X^i_{t+\delta t} = \cancel{F^i_{t+\delta t}(X', t)} + \textcolor{red}{T^i_{t+\delta t}(X')},
\end{align}
$$

where $T^i_{t+\delta t}(X')$ is a generator of the next state to occupy. This generator uses the current state transition probabilities (which are generally conditional on $X'$) at each new step.

**Summary of desirable features**

- using the learnings from the previous sections looking at specific example processes
- above formalism is so general that it can do anything - so while it shall serve as a useful guide and reference point, it would be good here to go through more of the specific desirable components we want to have access to in the software itself
- it might not always be convenient to have the windowed histories stored as S but some other varying quantity which can be used to construct S? take fractional brownian motion as an example of this! hence, need to provide more possible input histories into S
- want the timestep to have either exponentially-sampled lengths or fixed lengths in time
- formalism already isn't explicit about the choice of deterministic integrator in time
- but also want to be able to choose the stochastic integrator in continuous processes (Itô or Stratonovich?)
- enable correlated noise terms at the sample generator level
- configurable setup of simulations with just yamls + a single .go file defining the terms

**Design choices for the stochadex**